# 🧪 Feature Engineering

El *feature engineering* es el proceso de transformar, crear o seleccionar variables (features) para mejorar el rendimiento de los modelos de machine learning.

🔹 A veces los datos originales no contienen directamente las variables que más ayudan al modelo.  
🔹 Crear buenas features puede marcar más diferencia que cambiar de modelo.

En este notebook, trabajaremos sobre el dataset del Titanic para aplicar:

- Selección y transformación de variables
- Codificación de variables categóricas
- Generación de nuevas variables (features sintéticas)
- Discretización, escalado y más


## Importación de librerías

In [ ]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset("titanic")
df.head()

## Limpieza básica

In [ ]:
# Eliminar columnas poco útiles para el modelo
df = df.drop(columns=['deck', 'embark_town', 'alive'])

# Imputar valores nulos
df['age'] = df['age'].fillna(df['age'].median())
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])

## Códificación de variables categóricas

Label Encoding (para variables ordinales o binarias)

In [ ]:
df['sex'] = df['sex'].map({'male': 0, 'female': 1})

One-Hot Encoding (para variables nominales)

In [ ]:
df = pd.get_dummies(df, columns=['embarked'], drop_first=True)

## Crear nuevas variables

A. Título del pasajero (Mr, Mrs, Miss…)

In [ ]:
df['title'] = df['name'].str.extract('([A-Za-z]+)\.', expand=False)

# Agrupar títulos raros
df['title'] = df['title'].replace(['Lady', 'Countess', 'Capt', 'Col',
                                   'Don', 'Dr', 'Major', 'Rev', 'Sir',
                                   'Jonkheer', 'Dona'], 'Rare')

# Simplificar
df['title'] = df['title'].replace('Mlle', 'Miss')
df['title'] = df['title'].replace('Ms', 'Miss')
df['title'] = df['title'].replace('Mme', 'Mrs')

df['title'] = df['title'].map({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Rare': 4})

B. Tamaño de familia

In [ ]:
df['family_size'] = df['sibsp'] + df['parch'] + 1

C. ¿Está solo?

In [ ]:
df['is_alone'] = (df['family_size'] == 1).astype(int)

## Escalado de variables numéricas

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['age_scaled', 'fare_scaled']] = scaler.fit_transform(df[['age', 'fare']])

## Otras transformaciones útiles

Discretización (binning)

In [ ]:
# Agrupar edad en rangos
df['age_bin'] = pd.cut(df['age'], bins=[0, 12, 18, 35, 60, 100], labels=['child', 'teen', 'young_adult', 'adult', 'senior'])

## Modelo final

In [ ]:
df_model = df[['pclass', 'sex', 'age_scaled', 'fare_scaled', 'title', 'family_size', 'is_alone', 'embarked_Q', 'embarked_S', 'survived']]
df_model.head()

- Crear variables como `title` o `is_alone` puede capturar información útil que no se ve directamente.
- La codificación y escalado son pasos esenciales si usarás modelos que lo requieran (como SVM o redes neuronales).
- Siempre es bueno visualizar las nuevas variables para entender si agregan valor.
